# Abstract

Make the test comparison associations

# Environment

In [1]:
# Setup debugging log
import logging
logger = logging.getLogger('jwst.associations')
handler = logging.StreamHandler()
logger.addHandler(handler)
handler.setLevel(logging.DEBUG)
logger.setLevel(logging.DEBUG)

In [2]:
from glob import glob
from os import (
    listdir, 
    mkdir,
    path
)
import pdb
import re
import shutil
from tempfile import TemporaryDirectory

In [3]:
import numpy as np

In [4]:
from jwst.associations import (
    AssociationRegistry,
    generate
)
from jwst.associations.main import Main

In [5]:
from jwst.associations.tests.helpers import (
    combine_pools,
    registry_level3_only,
    registry_level2_only,
    t_path,
)

In [6]:
from jwst.associations.tests.test_standards import (
    DEF_ARGS,
    LV2_ONLY_ARGS,
    LV3_ONLY_ARGS,
    MakePars,
    standards
)

In [7]:
# Make all associations
MAKE_ALL = True

In [8]:
# Folders
DATA_DIR = path.join(
    '..', 'data'
)
NEW_ASN_DIR = path.join(
    DATA_DIR,
    'new_asn_standards'
)
SDP_DIR = path.join(
    '..', 'data', 'sdp'
)
SDP_POOLS = path.join(
    SDP_DIR, 'pools'
)
SDP_NEW_ASN_DIR = path.join(
    SDP_DIR, 'new_asns'
)

# Library

In [9]:
def make_standards(pool_root, 
                   main_args=None,
                   source=DATA_DIR,
                   outdir=NEW_ASN_DIR
                  ):
    """Make the association standards for a pool"""
    pool_path = path.join(
        source,
        pool_root + '.csv'
    )
    pool = combine_pools(pool_path)
    if main_args is None:
        main_args = []
    with TemporaryDirectory() as tmp_path:
        args = main_args + ['-v', '-p', tmp_path]
        results = Main(
            args,
            pool=pool
        )
        asn_paths = listdir(tmp_path)
        for asn_path in asn_paths:
            matches = re.match('[^-]+(-.+)', asn_path)
            asn_tail = matches.group(1)
            from_path = path.join(tmp_path, asn_path)
            to_path = path.join(outdir, pool_root + asn_tail)
            shutil.move(from_path, to_path)
        pool.write(path.join(outdir, pool_root + '_std.csv'))

# Main

## Clear result folder

In [10]:
try:
    shutil.rmtree(NEW_ASN_DIR)
except FileNotFoundError:
    pass

mkdir(NEW_ASN_DIR)

In [11]:
try:
    shutil.rmtree(SDP_NEW_ASN_DIR)
except FileNotFoundError:
    pass

mkdir(SDP_NEW_ASN_DIR)

## Make the standards

In [12]:
for standard in standards:
    if MAKE_ALL:
        make_standards(
            standard.pool_root,
            main_args=standard.main_args,
            source=DATA_DIR,
            outdir=NEW_ASN_DIR
        )

Command-line arguments: ['-r', '/Users/eisenham/anaconda3/envs/jwstdev/lib/python3.6/site-packages/jwst-0.7.8rc9.dev124-py3.6-macosx-10.7-x86_64.egg/jwst/associations/tests/../lib/rules_level3.py', '--ignore-default', '-v', '-p', '/var/folders/z7/hcykr_n546s_5033t9rj5d200000zj/T/tmp_3wpm3lr']
Reading rules.
Generating associations.
There where 1 associations and 0 orphaned items found.
Associations found are:
jw99009-o001_image3_001_asn with 1 products
Rule=candidate_Asn_Image
Constraints:
    wfsvisit: {'prime_targeted_fixed'}
    exp_type: {'mir_image'}
    is_not_tso: set()
    opt_elem: {'f560w'}
    opt_elem2: set()
    target: {'1'}
    program: {'99009'}
    instrument: {'miri'}
    asn_candidate: {"\\(\\'o001\\'\\,\\ \\'observation\\'\\)"}
Products:
	jw99009-o001_t001_miri_f560w with 8 members
Command-line arguments: ['-v', '-p', '/var/folders/z7/hcykr_n546s_5033t9rj5d200000zj/T/tmp83jzdhue']
Reading rules.
Generating associations.
There where 2 associations and 0 orphaned item

Reading rules.
Generating associations.
There where 17 associations and 0 orphaned items found.
Associations found are:
jw99009-a3001_spec3_001_asn with 1 products
Rule=discover_Asn_MIRI_LRS_FIXEDSLIT
Constraints:
    exp_type: {'mir_lrs\\-fixedslit', 'mir_tacq'}
    opt_elem: {'p750l'}
    subarray: {'full'}
    is_not_tso: set()
    instrument: {'miri'}
    target: {'1'}
    program: {'99009'}
Products:
	jw99009-a3001_t001_miri_p750l with 3 members
jw99009-a3001_spec3_002_asn with 1 products
Rule=discover_Asn_MIRI_IFU
Constraints:
    exp_type: {'mir_flatmrs', 'mir_mrs', 'mir_tacq'}
    is_not_tso: set()
    instrument: {'miri'}
    target: {'1'}
    program: {'99009'}
Products:
	jw99009-a3001_t001_miri with 39 members
jw99009-a3001_spec3_003_asn with 1 products
Rule=discover_Asn_MIRI_LRS_SLITLESS
Constraints:
    exp_type: {'mir_lrs\\-slitless', 'mir_tacq'}
    opt_elem: {'p750l'}
    subarray: {'subprism'}
    is_not_tso: set()
    instrument: {'miri'}
    target: {'1'}
    program

201712041015:WARNING:jwst.associations.lib.rules_level2_base.rename_to_level2a:Item FILENAME="set acid" is not a Level 1b name. Cannot transform to Level 2a.
201712041015:WARNING:jwst.associations.lib.rules_level2_base.rename_to_level2a:Item FILENAME="set acid" is not a Level 1b name. Cannot transform to Level 2a.
201712041015:WARNING:jwst.associations.lib.rules_level2_base.rename_to_level2a:Item FILENAME="set acid" is not a Level 1b name. Cannot transform to Level 2a.
201712041015:WARNING:jwst.associations.lib.rules_level2_base.rename_to_level2a:Item FILENAME="set acid" is not a Level 1b name. Cannot transform to Level 2a.
201712041015:WARNING:jwst.associations.lib.rules_level2_base.rename_to_level2a:Item FILENAME="set acid" is not a Level 1b name. Cannot transform to Level 2a.
201712041015:WARNING:jwst.associations.lib.rules_level2_base.rename_to_level2a:Item FILENAME="set acid" is not a Level 1b name. Cannot transform to Level 2a.
201712041015:WARNING:jwst.associations.lib.rules_lev

There where 2 associations and 27 orphaned items found.
Associations found are:
Association jw99009-o001_spec2_001_asn
    Parameters:
        Product type: spec2
        Rule:         candidate_Asn_Lv2Spec
        Program:      99009
        Target:       1
        Pool:         pool_009_spec_miri_lv2bkg
        Constraints:
            single_science: {'False'}
            exp_type: {'mir_lrs\\-fixedslit'}
            program: {'99009'}
            target: {'1'}
            instrument: {'miri'}
            detector: {'mirimage'}
            opt_elem: {'p750l'}
            opt_elem2: set()
            subarray: {'full'}
            channel: set()
            asn_candidate: {"\\(\\'o001\\'\\,\\ \\'observation\\'\\)"}
            is_tso: set()
	jw_00001 with 2 members
	jw_00002 with 1 members
	jw_00004 with 2 members
	jw_00008 with 1 members
	jw_00009 with 2 members
	jw_00012 with 1 members
	jw_00014 with 1 members
	jw_00029 with 2 members
	jw_00039 with 1 members


Association jw99009-

201712041015:WARNING:jwst.associations.lib.rules_level2_base.rename_to_level2a:Item FILENAME="set acid" is not a Level 1b name. Cannot transform to Level 2a.
201712041015:WARNING:jwst.associations.lib.rules_level2_base.rename_to_level2a:Item FILENAME="set acid" is not a Level 1b name. Cannot transform to Level 2a.
201712041015:WARNING:jwst.associations.lib.rules_level2_base.rename_to_level2a:Item FILENAME="set acid" is not a Level 1b name. Cannot transform to Level 2a.
201712041015:WARNING:jwst.associations.lib.rules_level2_base.rename_to_level2a:Item FILENAME="set acid" is not a Level 1b name. Cannot transform to Level 2a.
201712041015:WARNING:jwst.associations.lib.rules_level2_base.rename_to_level2a:Item FILENAME="set acid" is not a Level 1b name. Cannot transform to Level 2a.
201712041015:WARNING:jwst.associations.lib.rules_level2_base.rename_to_level2a:Item FILENAME="set acid" is not a Level 1b name. Cannot transform to Level 2a.
201712041015:WARNING:jwst.associations.lib.rules_lev

...

201712041015:WARNING:jwst.associations.lib.rules_level2_base.rename_to_level2a:Item FILENAME="set acid" is not a Level 1b name. Cannot transform to Level 2a.
201712041015:WARNING:jwst.associations.lib.rules_level2_base.rename_to_level2a:Item FILENAME="set acid" is not a Level 1b name. Cannot transform to Level 2a.
201712041015:WARNING:jwst.associations.lib.rules_level2_base.rename_to_level2a:Item FILENAME="set acid" is not a Level 1b name. Cannot transform to Level 2a.
201712041015:WARNING:jwst.associations.lib.rules_level2_base.rename_to_level2a:Item FILENAME="set acid" is not a Level 1b name. Cannot transform to Level 2a.
201712041015:WARNING:jwst.associations.lib.rules_level2_base.rename_to_level2a:Item FILENAME="set acid" is not a Level 1b name. Cannot transform to Level 2a.
201712041015:WARNING:jwst.associations.lib.rules_level2_base.rename_to_level2a:Item FILENAME="set acid" is not a Level 1b name. Cannot transform to Level 2a.
201712041015:WARNING:jwst.associations.lib.rules_lev

There where 7 associations and 2 orphaned items found.
Associations found are:
jw99009-o002_tso3_001_asn with 1 products
Rule=candidate_Asn_TSO_Flag
Constraints:
    is_tso: {'t'}
    exp_type: {'mir_lrs\\-slitless'}
    opt_elem: {'f140m'}
    opt_elem2: set()
    target: {'1'}
    program: {'99009'}
    instrument: {'miri'}
    asn_candidate: {"\\(\\'o002\\'\\,\\ \\'observation\\'\\)"}
Products:
	jw99009-o002_t001_miri_f140m with 1 members
jw99009-o004_tso3_001_asn with 1 products
Rule=candidate_Asn_TSO_Flag
Constraints:
    is_tso: {'t'}
    exp_type: {'nis_soss'}
    opt_elem: {'f140m'}
    opt_elem2: set()
    target: {'1'}
    program: {'99009'}
    instrument: {'niriss'}
    asn_candidate: {"\\(\\'o004\\'\\,\\ \\'observation\\'\\)"}
Products:
	jw99009-o004_t001_niriss_f140m with 1 members
jw99009-o006_tso3_001_asn with 1 products
Rule=candidate_Asn_TSO_Flag
Constraints:
    is_tso: {'t'}
    exp_type: {'nrc_tsimage'}
    opt_elem: {'f140m'}
    opt_elem2: set()
    target: {'1'}

201712041015:WARNING:jwst.associations.lib.rules_level2_base.rename_to_level2a:Item FILENAME="set acid" is not a Level 1b name. Cannot transform to Level 2a.
201712041015:WARNING:jwst.associations.lib.rules_level2_base.rename_to_level2a:Item FILENAME="set acid" is not a Level 1b name. Cannot transform to Level 2a.
201712041015:WARNING:jwst.associations.lib.rules_level2_base.rename_to_level2a:Item FILENAME="set acid" is not a Level 1b name. Cannot transform to Level 2a.
201712041015:WARNING:jwst.associations.lib.rules_level2_base.rename_to_level2a:Item FILENAME="set acid" is not a Level 1b name. Cannot transform to Level 2a.
201712041015:WARNING:jwst.associations.lib.rules_level2_base.rename_to_level2a:Item FILENAME="set acid" is not a Level 1b name. Cannot transform to Level 2a.
201712041015:WARNING:jwst.associations.lib.rules_level2_base.rename_to_level2a:Item FILENAME="set acid" is not a Level 1b name. Cannot transform to Level 2a.
201712041015:WARNING:jwst.associations.lib.rules_lev

201712041015:WARNING:jwst.associations.lib.rules_level2_base.rename_to_level2a:Item FILENAME="set acid" is not a Level 1b name. Cannot transform to Level 2a.
201712041015:WARNING:jwst.associations.lib.rules_level2_base.rename_to_level2a:Item FILENAME="set acid" is not a Level 1b name. Cannot transform to Level 2a.
201712041015:WARNING:jwst.associations.lib.rules_level2_base.rename_to_level2a:Item FILENAME="set acid" is not a Level 1b name. Cannot transform to Level 2a.
201712041015:WARNING:jwst.associations.lib.rules_level2_base.rename_to_level2a:Item FILENAME="set acid" is not a Level 1b name. Cannot transform to Level 2a.
201712041015:WARNING:jwst.associations.lib.rules_level2_base.rename_to_level2a:Item FILENAME="set acid" is not a Level 1b name. Cannot transform to Level 2a.
201712041015:WARNING:jwst.associations.lib.rules_level2_base.rename_to_level2a:Item FILENAME="set acid" is not a Level 1b name. Cannot transform to Level 2a.
201712041015:WARNING:jwst.associations.lib.rules_lev

There where 7 associations and 2 orphaned items found.
Associations found are:
jw99009-o002_tso3_001_asn with 1 products
Rule=candidate_Asn_TSO_EXPTYPE
Constraints:
    exp_type: {'mir_lrs\\-slitless'}
    no_tso_flag: Is Invalid
    opt_elem: {'f140m'}
    opt_elem2: set()
    target: {'1'}
    program: {'99009'}
    instrument: {'miri'}
    asn_candidate: {"\\(\\'o002\\'\\,\\ \\'observation\\'\\)"}
Products:
	jw99009-o002_t001_miri_f140m with 1 members
jw99009-o004_tso3_001_asn with 1 products
Rule=candidate_Asn_TSO_EXPTYPE
Constraints:
    exp_type: {'nis_soss'}
    no_tso_flag: Is Invalid
    opt_elem: {'f140m'}
    opt_elem2: set()
    target: {'1'}
    program: {'99009'}
    instrument: {'niriss'}
    asn_candidate: {"\\(\\'o004\\'\\,\\ \\'observation\\'\\)"}
Products:
	jw99009-o004_t001_niriss_f140m with 1 members
jw99009-o006_tso3_001_asn with 1 products
Rule=candidate_Asn_TSO_EXPTYPE
Constraints:
    exp_type: {'nrc_tsimage'}
    no_tso_flag: Is Invalid
    opt_elem: {'f140m'}

## Make the SPD-based associations

In [13]:
pools = glob(path.join(SDP_POOLS, '*.csv'))

In [14]:
for pool in pools:
    if MAKE_ALL:
        pool_dir, pool = path.split(pool)
        pool, ext = path.splitext(pool)
        make_standards(
            pool,
            source=pool_dir,
            outdir=SDP_NEW_ASN_DIR
        )

Command-line arguments: ['-v', '-p', '/var/folders/z7/hcykr_n546s_5033t9rj5d200000zj/T/tmpjcftwa6_']
Reading rules.
Generating associations.
There where 1 associations and 0 orphaned items found.
Associations found are:
Association jw00328-o001_image2_001_asn
    Parameters:
        Product type: image2
        Rule:         candidate_Asn_Lv2Image
        Program:      00328
        Target:       1
        Pool:         jw00328_20171108T101400_pool
        Constraints:
            single_science: {'False'}
            exp_type: {'nrc_image'}
            program: {'328'}
            target: {'1'}
            instrument: {'nircam'}
            detector: {'nrca2'}
            opt_elem: {'wlp4'}
            opt_elem2: {'wlm8'}
            subarray: {'full'}
            channel: {'short'}
            asn_candidate: {"\\(\\'o001\\'\\,\\ \\'observation\\'\\)"}
            is_tso: set()
	jw00328001001_02104_00001_nrca2 with 1 members
	jw00328001001_02104_00001_nrca1 with 1 members
	jw003280010

Command-line arguments: ['-v', '-p', '/var/folders/z7/hcykr_n546s_5033t9rj5d200000zj/T/tmp6_x27sd2']
Reading rules.
Generating associations.
There where 10 associations and 35 orphaned items found.
Associations found are:
jw80600-o006_image3_001_asn with 1 products
Rule=candidate_Asn_Image
Constraints:
    wfsvisit: {'prime_targeted_fixed'}
    exp_type: {'mir_image'}
    is_not_tso: set()
    opt_elem: {'f2100w'}
    opt_elem2: set()
    target: {'1'}
    program: {'80600'}
    instrument: {'miri'}
    asn_candidate: {"\\(\\'o006\\'\\,\\ \\'observation\\'\\)"}
Products:
	jw80600-o006_t001_miri_f2100w with 4 members
jw80600-o013_spec3_001_asn with 1 products
Rule=candidate_Asn_MIRI_LRS_FIXEDSLIT
Constraints:
    exp_type: {'mir_lrs\\-fixedslit'}
    opt_elem: {'p750l'}
    subarray: {'full'}
    is_not_tso: set()
    instrument: {'miri'}
    target: {'1'}
    program: {'80600'}
    asn_candidate: {"\\(\\'o013\\'\\,\\ \\'observation\\'\\)"}
Products:
	jw80600-o013_t001_miri_p750l with 1

Command-line arguments: ['-v', '-p', '/var/folders/z7/hcykr_n546s_5033t9rj5d200000zj/T/tmpswvfng91']
Reading rules.
Generating associations.
There where 18 associations and 0 orphaned items found.
Associations found are:
jw84600-a3001_spec3_001_asn with 1 products
Rule=discover_Asn_NRS_IFU
Constraints:
    exp_type: {'nrs_confirm', 'nrs_ifu'}
    is_not_tso: set()
    instrument: {'nirspec'}
    target: {'1'}
    program: {'84600'}
Products:
	jw84600-a3001_t001_nirspec_clear with 10 members
jw84600-a3001_tso3_001_asn with 1 products
Rule=discover_Asn_TSO_EXPTYPE
Constraints:
    exp_type: {'nrs_brightobj'}
    no_tso_flag: Is Invalid
    opt_elem: {'f070lp'}
    opt_elem2: {'g140h'}
    target: {'1'}
    program: {'84600'}
    instrument: {'nirspec'}
Products:
	jw84600-a3001_t001_nirspec_f070lp-g140h with 3 members
jw84600-a3001_tso3_002_asn with 1 products
Rule=discover_Asn_TSO_EXPTYPE
Constraints:
    exp_type: {'nrs_brightobj'}
    no_tso_flag: Is Invalid
    opt_elem: {'f170lp'}
  

Command-line arguments: ['-v', '-p', '/var/folders/z7/hcykr_n546s_5033t9rj5d200000zj/T/tmpihn3i873']
Reading rules.
Generating associations.
There where 12 associations and 0 orphaned items found.
Associations found are:
jw84600-a3001_spec3_001_asn with 1 products
Rule=discover_Asn_NRS_IFU
Constraints:
    exp_type: {'nrs_confirm', 'nrs_ifu'}
    is_not_tso: set()
    instrument: {'nirspec'}
    target: {'1'}
    program: {'84600'}
Products:
	jw84600-a3001_t001_nirspec_clear with 10 members
jw84600-a3001_tso3_001_asn with 1 products
Rule=discover_Asn_TSO_EXPTYPE
Constraints:
    exp_type: {'nrs_brightobj'}
    no_tso_flag: Is Invalid
    opt_elem: {'f070lp'}
    opt_elem2: {'g140h'}
    target: {'1'}
    program: {'84600'}
    instrument: {'nirspec'}
Products:
	jw84600-a3001_t001_nirspec_f070lp-g140h with 3 members
jw84600-a3001_spec3_002_asn with 1 products
Rule=discover_Asn_NRS_MSA
Constraints:
    exp_type: {'nrs_msaspec', 'nrs_autoflat'}
    is_not_tso: set()
    instrument: {'nirs

Command-line arguments: ['-v', '-p', '/var/folders/z7/hcykr_n546s_5033t9rj5d200000zj/T/tmpe6k1eub6']
Reading rules.
Generating associations.
There where 1 associations and 22 orphaned items found.
Associations found are:
Association jw86600-o001_image2_001_asn
    Parameters:
        Product type: image2
        Rule:         candidate_Asn_Lv2FGS
        Program:      86600
        Target:       1
        Pool:         jw86600_20171108T043532_pool
        Constraints:
            exp_type: {'fgs_focus'}
            single_science: {'False'}
            asn_candidate: {"\\(\\'o001\\'\\,\\ \\'observation\\'\\)"}
            program: {'86600'}
            is_tso: set()
	jw86600001001_02101_00005_guider1 with 1 members
	jw86600001001_02101_00003_guider1 with 1 members
	jw86600001001_02101_00002_guider1 with 1 members
	jw86600001001_02101_00001_guider1 with 1 members
	jw86600016001_02101_00001_guider1 with 1 members
	jw86600003001_02101_00001_guider2 with 1 members
	jw86600012001_02101_0000

...

Command-line arguments: ['-v', '-p', '/var/folders/z7/hcykr_n546s_5033t9rj5d200000zj/T/tmptcdo4z0d']
Reading rules.
Generating associations.
There where 6 associations and 0 orphaned items found.
Associations found are:
jw93135-a3001_spec3_001_asn with 1 products
Rule=discover_Asn_MIRI_IFU
Constraints:
    exp_type: {'mir_flatmrs', 'mir_mrs'}
    is_not_tso: set()
    instrument: {'miri'}
    target: {'2'}
    program: {'93135'}
Products:
	jw93135-a3001_t002_miri with 48 members
jw93135-a3001_spec3_002_asn with 1 products
Rule=discover_Asn_MIRI_IFU
Constraints:
    exp_type: {'mir_flatmrs', 'mir_mrs'}
    is_not_tso: set()
    instrument: {'miri'}
    target: {'1'}
    program: {'93135'}
Products:
	jw93135-a3001_t001_miri with 44 members
jw93135-a3001_spec3_003_asn with 1 products
Rule=discover_Asn_MIRI_IFU
Constraints:
    exp_type: {'mir_flatmrs', 'mir_mrs'}
    is_not_tso: set()
    instrument: {'miri'}
    target: {'5'}
    program: {'93135'}
Products:
	jw93135-a3001_t005_miri with

Command-line arguments: ['-v', '-p', '/var/folders/z7/hcykr_n546s_5033t9rj5d200000zj/T/tmpl25c9fto']
Reading rules.
Generating associations.
There where 8 associations and 0 orphaned items found.
Associations found are:
jw95065-o009_spec3_001_asn with 1 products
Rule=candidate_Asn_NRS_MSA
Constraints:
    exp_type: {'nrs_msaspec'}
    is_not_tso: set()
    instrument: {'nirspec'}
    opt_elem: {'f100lp'}
    opt_elem2: {'g140m'}
    target: {'2'}
    program: {'95065'}
    asn_candidate: {"\\(\\'o009\\'\\,\\ \\'observation\\'\\)"}
Products:
	jw95065-o009_t002_nirspec_f100lp-g140m with 6 members
jw95065-o009_spec3_002_asn with 1 products
Rule=candidate_Asn_NRS_MSA
Constraints:
    exp_type: {'nrs_msaspec'}
    is_not_tso: set()
    instrument: {'nirspec'}
    opt_elem: {'f290lp'}
    opt_elem2: {'g395m'}
    target: {'2'}
    program: {'95065'}
    asn_candidate: {"\\(\\'o009\\'\\,\\ \\'observation\\'\\)"}
Products:
	jw95065-o009_t002_nirspec_f290lp-g395m with 6 members
jw95065-o009_spe

Command-line arguments: ['-v', '-p', '/var/folders/z7/hcykr_n546s_5033t9rj5d200000zj/T/tmp9s_xzcwr']
Reading rules.
Generating associations.
There where 3 associations and 0 orphaned items found.
Associations found are:
jw96090-o001_image3_001_asn with 1 products
Rule=candidate_Asn_Image
Constraints:
    wfsvisit: {'prime_targeted_fixed'}
    exp_type: {'nrc_image'}
    is_not_tso: set()
    opt_elem: {'f150w2'}
    opt_elem2: {'clear'}
    target: {'1'}
    program: {'96090'}
    instrument: {'nircam'}
    asn_candidate: {"\\(\\'o001\\'\\,\\ \\'observation\\'\\)"}
Products:
	jw96090-o001_t001_nircam_f150w2 with 32 members
jw96090-o001_image3_002_asn with 1 products
Rule=candidate_Asn_Image
Constraints:
    wfsvisit: {'prime_targeted_fixed'}
    exp_type: {'nrc_image'}
    is_not_tso: set()
    opt_elem: {'f322w2'}
    opt_elem2: {'f323n'}
    target: {'1'}
    program: {'96090'}
    instrument: {'nircam'}
    asn_candidate: {"\\(\\'o001\\'\\,\\ \\'observation\\'\\)"}
Products:
	jw9609


Command-line arguments: ['-v', '-p', '/var/folders/z7/hcykr_n546s_5033t9rj5d200000zj/T/tmpl4u0j3lj']
Reading rules.
Generating associations.
There where 1 associations and 0 orphaned items found.
Associations found are:
Association jw98005-o001_spec2_001_asn
    Parameters:
        Product type: spec2
        Rule:         candidate_Asn_Lv2Spec
        Program:      98005
        Target:       1
        Pool:         jw98005_20171108T041409_pool
        Constraints:
            single_science: {'False'}
            exp_type: {'nrs_fixedslit'}
            program: {'98005'}
            target: {'1'}
            instrument: {'nirspec'}
            detector: {'nrs1'}
            opt_elem: {'f170lp'}
            opt_elem2: {'g235m'}
            subarray: {'s200a1'}
            channel: set()
            asn_candidate: {"\\(\\'o001\\'\\,\\ \\'observation\\'\\)"}
            is_tso: set()
	jw98005001001_0212o_00001_nrs1 with 1 members
	jw98005001001_0211m_00001_nrs1 with 1 members
	jw980050

Command-line arguments: ['-v', '-p', '/var/folders/z7/hcykr_n546s_5033t9rj5d200000zj/T/tmpxd5x2lta']
Reading rules.
Generating associations.
There where 2 associations and 0 orphaned items found.
Associations found are:
jw98010-o001_spec3_001_asn with 1 products
Rule=candidate_Asn_MIRI_LRS_FIXEDSLIT
Constraints:
    exp_type: {'mir_lrs\\-fixedslit'}
    opt_elem: {'p750l'}
    subarray: {'full'}
    is_not_tso: set()
    instrument: {'miri'}
    target: {'1'}
    program: {'98010'}
    asn_candidate: {"\\(\\'o001\\'\\,\\ \\'observation\\'\\)"}
Products:
	jw98010-o001_t001_miri_p750l with 4 members
Association jw98010-o001_spec2_001_asn
    Parameters:
        Product type: spec2
        Rule:         candidate_Asn_Lv2Spec
        Program:      98010
        Target:       1
        Pool:         jw98010_20171108T062332_pool
        Constraints:
            single_science: {'False'}
            exp_type: {'mir_lrs\\-fixedslit'}
            program: {'98010'}
            target: {'1'}
   